In [ ]:
import torch
import torch.nn as nn


class SudokuPositionalEncoding(nn.Module):
    """
    数独専用のPositional Encoding付きInput Projection
    行、列、3x3ブロックの位置情報を学習可能なembeddingとして追加
    """

    def __init__(self, C, D, bias=False):
        """
        Args:
            C: 入力次元（数独の場合は9）
            D: 出力次元（隠れ層の次元）
            bias: Linear層でbiasを使うかどうか
        """
        super().__init__()

        # 元のLinear投影層
        self.linear = nn.Linear(C, D, bias=bias)

        # 位置埋め込み（行、列、ブロックそれぞれ9個）
        self.row_embedding = nn.Parameter(torch.randn(9, D) * 0.02)
        self.col_embedding = nn.Parameter(torch.randn(9, D) * 0.02)
        self.block_embedding = nn.Parameter(torch.randn(9, D) * 0.02)

        # positional encodingのスケール係数（学習可能）
        self.pos_scale = nn.Parameter(torch.tensor(0.1))

        # パラメータ情報を保持（互換性のため）
        self.in_features = C
        self.out_features = D
        self.weight = self.linear.weight  # 元のlinearの重みへの参照
        if bias:
            self.bias = self.linear.bias

    def forward(self, X):
        """
        Args:
            X: 入力テンソル (B, T=81, C=9)

        Returns:
            位置エンコーディング付きの出力 (B, T=81, D)
        """
        B, T, C = X.shape
        device = X.device

        # Linear投影
        X_proj = self.linear(X)  # (B, T, D)

        # 各セルの位置エンコーディングを構築
        pos_encoding = []
        for idx in range(81):
            row = idx // 9
            col = idx % 9
            block = (row // 3) * 3 + (col // 3)

            # 3つの位置埋め込みを加算
            pos = self.row_embedding[row] + self.col_embedding[col] + self.block_embedding[block]
            pos_encoding.append(pos)

        # テンソルにまとめてバッチ次元を追加
        pos_encoding = torch.stack(pos_encoding).unsqueeze(0)  # (1, 81, D)
        pos_encoding = pos_encoding.expand(B, -1, -1).to(device)

        # 投影結果と位置エンコーディングを加算
        output = X_proj + self.pos_scale * pos_encoding

        return output


def add_sudoku_positional_encoding(model):
    """
    既存のDEQモデルのinput_projをSudoku用positional encoding付きに置き換える

    Args:
        model: StaticDEQ, HierarchicalDEQ, またはHyperDEQのインスタンス

    Returns:
        input_projが置き換えられた同じモデルインスタンス
    """

    # 元のinput_projのパラメータを取得
    old_input_proj = model.input_proj
    C = old_input_proj.in_features
    D = old_input_proj.out_features
    bias = old_input_proj.bias is not None

    # 新しいSudoku用投影層を作成
    new_input_proj = SudokuPositionalEncoding(C, D, bias=bias)

    # 元のLinear層の重みをコピー（学習済みモデルの場合に重要）
    with torch.no_grad():
        new_input_proj.linear.weight.copy_(old_input_proj.weight)
        if bias:
            new_input_proj.linear.bias.copy_(old_input_proj.bias)

    # モデルのinput_projを置き換え
    model.input_proj = new_input_proj

    # デバイスを合わせる
    if next(model.parameters()).is_cuda:
        model.input_proj = model.input_proj.cuda()

    return model